**Soovitatav on käivitada kõik järgnev Google Colab'is. Teistel platvormidel ei saa garanteerida, et kõik toimib nii nagu peaks.**

In [ ]:
# Colabi jaoks
from google.colab import drive
from google.colab import runtime
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Lõputöö/Bakalaureus') # Muutke oma Drive asukohaks

Mounted at /content/drive


In [1]:
!python --version

Python 3.10.12


In [2]:
%pip install estnltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.2/222.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.9 MB/s eta 0:00:00


In [3]:
# Importimised
import random
import csv
import nltk
import estnltk
import pandas as pd
import numpy as np
import sklearn as sk

In [4]:
print(f"nltk: {nltk.__version__}") #
print(f"estnltk: {estnltk.__version__}") #
print(f"pandas: {pd.__version__}") # pandas: 2.0.3
print(f"numpy: {np.__version__}") # numpy: 1.25.2
print(f"sklearn: {sk.__version__}") # sklearn: 1.2.2

nltk: 3.8.1
estnltk: 1.7.2
pandas: 2.0.3
numpy: 1.25.2
sklearn: 1.2.2


## Süntees

Selles peatükis sünteesitakse algvormidele kõik muutevormid, seejärel kogutakse kokku ühte andmestikku, mida lõpuks salvestatakse *.csv* faili.

### Ettevalmistus

Järgmine info on võetud [EstNLTK morfoloogiliste kategooriate tabelitest](https://github.com/estnltk/estnltk/blob/4236f2033110d2bf20fc7f565950c0a2170f8573/tutorials/nlp_pipeline/B_morphology/00_tables_of_morphological_categories.ipynb)

Käänete kirjeldused:

* `ab`    abessiiv (ilmaütlev)
* `abl`   ablatiiv (alaltütlev)
* `ad`    adessiiv (alalütlev)
* `adt`   aditiiv suunduv (lühike sisseütlev), nt. "majja"
* `all`   allatiiv (alaleütlev)
* `el`    elatiiv (seestütlev)
* `es`    essiiv (olev)
* `g`     genitiiv (omastav)
* `ill`   illatiiv (sisseütlev)
* `in`    inessiiv (seesütlev)
* `kom`   komitatiiv (kaasaütlev)
* `n`     nominatiiv (nimetav)
* `p`     partitiiv (osastav)
* `pl`    pluural (mitmus)
* `sg`    ainsus (ainsus)
* `ter`   terminatiiv (rajav)
* `tr`    translatiiv (saav)

Pöörete kirjeldused:

* `b`       kindel kõneviis olevik 3. isik ainsus aktiiv jaatav kõne 	loeb
* `d`       kindel kõneviis olevik 2. isik ainsus aktiiv jaatav kõne 	loed
* `da`      infinitiiv jaatav kõne 	lugeda
* `des`     gerundium jaatav kõne 	lugedes
* `ge`      käskiv kõneviis olevik 2. isik mitmus aktiiv jaatav kõne 	lugege
* `gem`     käskiv kõneviis olevik 1. isik mitmus aktiiv jaatav kõne 	lugegem
* `gu`      käskiv kõneviis olevik 3. isik mitmus aktiiv jaatav kõne 	(nad) lugegu
* `gu`      käskiv kõneviis olevik 3. isik ainsus aktiiv jaatav kõne 	(ta) lugegu
* `ks`      tingiv kõneviis olevik 1. isik mitmus aktiiv jaatav kõne 	(me) loeks
* `ks`      tingiv kõneviis olevik 1. isik ainsus aktiiv jaatav kõne 	(ma) loeks
* `ks`      tingiv kõneviis olevik 2. isik mitmus aktiiv jaatav kõne 	(te) loeks
* `ks`      tingiv kõneviis olevik 2. isik ainsus aktiiv jaatav kõne 	(sa) loeks
* `ks`      tingiv kõneviis olevik 3. isik mitmus aktiiv jaatav kõne 	(nad) loeks
* `ks`      tingiv kõneviis olevik 3. isik ainsus aktiiv jaatav kõne 	(ta) loeks
* `ksid`    tingiv kõneviis olevik 2. isik ainsus aktiiv jaatav kõne 	(sa) loeksid
* `ksid`    tingiv kõneviis olevik 3. isik mitmus aktiiv jaatav kõne 	(nad) loeksid
* `ksime`   tingiv kõneviis olevik 1. isik mitmus aktiiv jaatav kõne 	(me) loeksime
* `ksin`    tingiv kõneviis olevik 1. isik ainsus aktiiv jaatav kõne 	(ma) loeksin
* `ksite`   tingiv kõneviis olevik 2. isik mitmus aktiiv jaatav kõne 	(te) loeksite
* `ma`      supiin aktiiv jaatav kõne sisseütlev 	lugema
* `maks`    supiin aktiiv jaatav kõne saav 	lugemaks
* `mas`     supiin aktiiv jaatav kõne seesütlev 	lugemas
* `mast`    supiin aktiiv jaatav kõne seestütlev 	lugemast
* `mata`    supiin aktiiv jaatav kõne ilmaütlev 	lugemata
* `me`      kindel kõneviis olevik 1. isik mitmus aktiiv jaatav kõne 	loeme
* `n`       kindel kõneviis olevik 1. isik ainsus aktiiv jaatav kõne 	loen
* `neg`     eitav kõne 	ei
* `neg`     ge 	käskiv kõneviis olevik 2. isik mitmus aktiiv eitav kõne 	ärge
* `neg`     gem 	käskiv kõneviis olevik 1. isik mitmus aktiiv eitav kõne 	ärgem
* `neg`     gu 	käskiv kõneviis olevik 3. isik mitmus aktiiv eitav kõne 	(nad) ärgu
* `neg`     gu 	käskiv kõneviis olevik 3. isik ainsus aktiiv eitav kõne 	(ta) ärgu
* `neg`     gu 	käskiv kõneviis olevik passiiv eitav kõne 	ärgu
* `neg`     ks 	tingiv kõneviis olevik 1. isik mitmus aktiiv eitav kõne 	(me) poleks
* `neg`     ks 	tingiv kõneviis olevik 1. isik ainsus aktiiv eitav kõne 	(ma) poleks
* `neg`     ks 	tingiv kõneviis olevik 2. isik mitmus aktiiv eitav kõne 	(te) poleks
* `neg`     ks 	tingiv kõneviis olevik 2. isik ainsus aktiiv eitav kõne 	(sa) poleks
* `neg`     ks 	tingiv kõneviis olevik 3. isik mitmus aktiiv eitav kõne 	(nad) poleks
* `neg`     ks 	tingiv kõneviis olevik 3. isik ainsus aktiiv eitav kõne 	(ta) poleks
* `neg`     me 	käskiv kõneviis olevik 1. isik mitmus aktiiv eitav kõne 	ärme
* `neg`     nud 	kindel kõneviis lihtminevik 1. isik mitmus aktiiv eitav kõne 	(me) polnud
* `neg`     nud 	kindel kõneviis lihtminevik 1. isik ainsus aktiiv eitav kõne 	(ma) polnud
* `neg`     nud 	kindel kõneviis lihtminevik 2. isik mitmus aktiiv eitav kõne 	(te) polnud
* `neg`     nud 	kindel kõneviis lihtminevik 2. isik ainsus aktiiv eitav kõne 	(sa) polnud
* `neg`     nud 	kindel kõneviis lihtminevik 3. isik mitmus aktiiv eitav kõne 	(nad) polnud
* `neg`     nud 	kindel kõneviis lihtminevik 3. isik ainsus aktiiv eitav kõne 	(ta) polnud
* `neg`     nuks 	tingiv kõneviis minevik 1. isik mitmus aktiiv eitav kõne 	(me) polnuks
* `neg`     nuks 	tingiv kõneviis minevik 1. isik ainsus aktiiv eitav kõne 	(ma) polnuks
* `neg`     nuks 	tingiv kõneviis minevik 2. isik mitmus aktiiv eitav kõne 	(te) polnuks
* `neg`     nuks 	tingiv kõneviis minevik 2. isik ainsus aktiiv eitav kõne 	(sa) polnuks
* `neg`     nuks 	tingiv kõneviis minevik 3. isik mitmus aktiiv eitav kõne 	(nad) polnuks
* `neg`     nuks 	tingiv kõneviis minevik 3. isik ainsus aktiiv eitav kõne 	(ta) polnuks
* `neg`     o 	käskiv kõneviis olevik 2. isik ainsus aktiiv eitav kõne 	ära
* `neg`     o 	kindel kõneviis olevik 1. isik mitmus aktiiv eitav kõne 	(me) pole
* `neg`     o 	kindel kõneviis olevik 1. isik ainsus aktiiv eitav kõne 	(ma) pole
* `neg`     o 	kindel kõneviis olevik 2. isik mitmus aktiiv eitav kõne 	(te) pole
* `neg`     o 	kindel kõneviis olevik 2. isik ainsus aktiiv eitav kõne 	(sa) pole
* `neg`     o 	kindel kõneviis olevik 3. isik mitmus aktiiv eitav kõne 	(nad) pole
* `neg`     o 	kindel kõneviis olevik 3. isik ainsus aktiiv eitav kõne 	(ta) pole
* `neg`     vat 	kaudne kõneviis olevik 1. isik mitmus aktiiv eitav kõne 	(me) polevat
* `neg`     vat 	kaudne kõneviis olevik 1. isik ainsus aktiiv eitav kõne 	(ma) polevat
* `neg`     tud 	kesksõna minevik passiiv eitav kõne 	poldud
* `neg`     vat 	kaudne kõneviis olevik 2. isik mitmus aktiiv eitav kõne 	(te) polevat
* `neg`     vat 	kaudne kõneviis olevik 2. isik ainsus aktiiv eitav kõne 	(sa) polevat
* `neg`     vat 	kaudne kõneviis olevik 3. isik mitmus aktiiv eitav kõne 	(nad) polevat
* `neg`     vat 	kaudne kõneviis olevik 3. isik ainsus aktiiv eitav kõne 	(ta) polevat
* `nud`     kesksõna minevik aktiiv jaatav kõne 	lugenud
* `nuks`    tingiv kõneviis minevik 1. isik mitmus aktiiv jaatav kõne 	(me) lugenuks
* `nuks`    tingiv kõneviis minevik 1. isik ainsus aktiiv jaatav kõne 	(ma) lugenuks
* `nuks`    tingiv kõneviis minevik 2. isik mitmus aktiiv jaatav kõne 	(te) lugenuks
* `nuks`    tingiv kõneviis minevik 2. isik ainsus aktiiv jaatav kõne 	(sa) lugenuks
* `nuks`    tingiv kõneviis minevik 3. isik mitmus aktiiv jaatav kõne 	(nad) lugenuks
* `nuks`    tingiv kõneviis minevik 3. isik ainsus aktiiv jaatav kõne 	(ta) lugenuks
* `nuksid`  tingiv kõneviis minevik 2. isik ainsus aktiiv jaatav kõne 	(sa) lugenuksid
* `nuksid`  tingiv kõneviis minevik 3. isik mitmus aktiiv jaatav kõne 	(nad) lugenuksid
* `nuksime` tingiv kõneviis minevik 1. isik mitmus aktiiv jaatav kõne 	lugenuksime
* `nuksin`  tingiv kõneviis minevik 1. isik ainsus aktiiv jaatav kõne 	lugenuksin
* `nuksite` tingiv kõneviis minevik 2. isik mitmus aktiiv jaatav kõne 	lugenuksite
* `nuvat`   kaudne kõneviis minevik 1. isik mitmus aktiiv jaatav kõne 	(me) lugenuvat
* `nuvat`   kaudne kõneviis minevik 1. isik ainsus aktiiv jaatav kõne 	(ma) lugenuvat
* `nuvat`   kaudne kõneviis minevik 2. isik mitmus aktiiv jaatav kõne 	(te) lugenuvat
* `nuvat`   kaudne kõneviis minevik 2. isik ainsus aktiiv jaatav kõne 	(sa) lugenuvat
* `nuvat`   kaudne kõneviis minevik 3. isik mitmus aktiiv jaatav kõne 	(nad) lugenuvat
* `nuvat`   kaudne kõneviis minevik 3. isik ainsus aktiiv jaatav kõne 	(ta) lugenuvat
* `o`       käskiv kõneviis olevik 2. isik ainsus aktiiv jaatav kõne 	loe
* `s`       kindel kõneviis lihtminevik 3. isik ainsus aktiiv jaatav kõne 	luges
* `sid`     kindel kõneviis lihtminevik 2. isik ainsus aktiiv jaatav kõne 	(sa) lugesid
* `sid`     kindel kõneviis lihtminevik 3. isik mitmus aktiiv jaatav kõne 	(nad) lugesid
* `sime`    kindel kõneviis lihtminevik 1. isik mitmus aktiiv jaatav kõne 	lugesime
* `sin`     kindel kõneviis lihtminevik 1. isik ainsus aktiiv jaatav kõne 	lugesin
* `site`    kindel kõneviis lihtminevik 2. isik mitmus aktiiv jaatav kõne 	lugesite
* `ta`      kindel kõneviis olevik passiiv eitav kõne 	loeta
* `tagu`    käskiv kõneviis olevik passiiv jaatav kõne 	loetagu
* `taks`    tingiv kõneviis olevik passiiv jaatav kõne 	loetaks
* `takse`   kindel kõneviis olevik passiiv jaatav kõne 	loetakse
* `tama`    supiin passiiv jaatav kõne 	loetama
* `tav`     kesksõna olevik passiiv jaatav kõne 	loetav
* `tavat`   kaudne kõneviis olevik passiiv jaatav kõne 	loetavat
* `te`      kindel kõneviis olevik 2. isik mitmus aktiiv jaatav kõne 	loete
* `ti`      kindel kõneviis lihtminevik passiiv jaatav kõne 	loeti
* `tud`     kesksõna minevik passiiv jaatav kõne 	loetud
* `tuks`    tingiv kõneviis minevik passiiv jaatav kõne 	loetuks
* `tuvat`   kaudne kõneviis minevik passiiv jaatav kõne 	loetuvat
* `v`       kesksõna olevik aktiiv jaatav kõne 	lugev
* `vad`     kindel kõneviis olevik 3. isik mitmus aktiiv jaatav kõne 	loevad
* `vat`     kaudne kõneviis olevik 1. isik mitmus aktiiv jaatav kõne 	(me) lugevat
* `vat`     kaudne kõneviis olevik 1. isik ainsus aktiiv jaatav kõne 	(ma) lugevat
* `vat`     kaudne kõneviis olevik 2. isik mitmus aktiiv jaatav kõne 	(te) lugevat
* `vat`     kaudne kõneviis olevik 2. isik ainsus aktiiv jaatav kõne 	(sa) lugevat
* `vat`     kaudne kõneviis olevik 3. isik mitmus aktiiv jaatav kõne 	(nad) lugevat
* `vat`     kaudne kõneviis olevik 3. isik ainsus aktiiv jaatav kõne 	(ta) lugevat

In [ ]:
# Järjendid andmete kogumiseks
sõnad = list()
sõnaliigid = list()
muuttüübid = list()
algvormid = list()

# Käänded - 14 * 2 = 28 + 1 (lühike sisseütlev) = 29
cases_noun = [
  # label, Estonian case name
  ('n', 'nimetav'),
  ('g', 'omastav'),
  ('p', 'osastav'),
  ('adt', 'lühike sisseütlev'),
  ('ill', 'sisseütlev'),
  ('in', 'seesütlev'),
  ('el', 'seestütlev'),
  ('all', 'alaleütlev'),
  ('ad', 'alalütlev'),
  ('abl', 'alaltütlev'),
  ('tr', 'saav'),
  ('ter', 'rajav'),
  ('es', 'olev'),
  ('ab', 'ilmaütlev'),
  ('kom', 'kaasaütlev')]

# Pöörded - 48
cases_verb = [
  'b',
  'd',
  'da',
  'des',
  'ge',
  'gem',
  'gu',
  'ks',
  'ksid',
  'ksime',
  'ksin',
  'ksite',
  'ma',
  'maks',
  'mas',
  'mast',
  'mata',
  'me',
  'n',
  'neg',
  'nud',
  'nuks',
  'nuksid',
  'nuksime',
  'nuksin',
  'nuksite',
  'nuvat',
  'o',
  's',
  'sid',
  'sime',
  'sin',
  'site',
  'ta',
  'tagu',
  'taks',
  'takse',
  'tama',
  'tav',
  'tavat',
  'te',
  'ti',
  'tud',
  'tuks',
  'tuvat',
  'v',
  'vad',
  'vat',
]

# Sõnaliigid
pos = {
  'A': 'N',
  'C': 'N',
  'D': 'K',
  'G': 'K',
  'H': 'N',
  'I': 'K',
  'J': 'K',
  'K': 'K',
  'N': 'N',
  'O': 'N',
  'P': 'N',
  'S': 'N',
  'U': 'N',
  'V': 'V',
  'X': 'N'
}

# Funktsioon kõikide muutevormide genereerimiseks ja visualiseerimiseks
def synthesize_all_visualize(word, PoS):
  match PoS:
    case 'N':
      case_rows = []
      sing_rows = []
      plur_rows = []
      for cas, case_name_est in cases_noun:
        case_rows.append( case_name_est )
        sing_rows.append(', '.join(estnltk.vabamorf.morf.synthesize(word, 'sg ' + cas)))
        plur_rows.append(', '.join(estnltk.vabamorf.morf.synthesize(word, 'pl ' + cas)))
      return pd.DataFrame({'case': case_rows, 'singular': sing_rows, 'plural': plur_rows}, columns=['case', 'singular', 'plural'])
    case 'V':
      case_rows = []
      sing_rows = []
      for cas in cases_verb:
        case_rows.append( cas )
        sing_rows.append(', '.join(estnltk.vabamorf.morf.synthesize(word, cas)))
      return pd.DataFrame({'case': case_rows, 'word': sing_rows}, columns=['case', 'word'])
    case _:
      ...

# Funktsioon kõikide muutevormide genereerimiseks
def synthesize_all(word, PoS):
  match PoS:
    case 'N':
      case_rows = []
      sing_rows = []
      plur_rows = []
      for cas, case_name_est in cases_noun:
        case_rows.append( case_name_est )
        sing_rows.append(estnltk.vabamorf.morf.synthesize(word, 'sg ' + cas))
        plur_rows.append(estnltk.vabamorf.morf.synthesize(word, 'pl ' + cas))
      return (sing_rows, plur_rows)
    case 'V':
      case_rows = []
      sing_rows = []
      for cas in cases_verb:
        case_rows.append( cas )
        sing_rows.append(estnltk.vabamorf.morf.synthesize(word, cas))
      return sing_rows
    case _:
      return None

# Funktsioon andmete lisamiseks uute andmestikku
def lisa_andmestikku(sõna, sõnaliik, muuttüüp, algvorm = None):
  global sõnad, sõnaliigid, muuttüübid, algvormid
  if algvorm:
    sõnad.append(sõna)
    sõnaliigid.append(sõnaliik)
    muuttüübid.append(muuttüüp)
    algvormid.append(algvorm)
  else:
    sõnad.append(sõna)
    sõnaliigid.append(sõnaliik)
    muuttüübid.append(muuttüüp)

#### Andmestiku kontrollimine

Enne sünteesimist kontrollitakse ja analüüsitakse andmestikku

In [ ]:
# Muuttüüp 0 andmestik
with open('muuttüübi_0_andmed.csv', 'r', encoding='utf-8') as muut_0_f:
  next(muut_0_f)
  for rida in muut_0_f:
    info = rida.strip().replace('"','').split(',')
    sõna = info[0]
    sõnaliik = []
    muuttüüp = []
    algvorm = info[-1]
    for i in info[1:-1]:
      if i.isalpha():
        sõnaliik.append(i)
        continue
      if i.isdecimal():
        muuttüüp.append(i)
        continue
    if len(sõnaliik) > 1 or len(muuttüüp) > 1: # Kontrolliks
      print(rida)
      print("Mitme sõnaliigi või muuttüübiga")
    lisa_andmestikku(sõna, sõnaliik, muuttüüp, algvorm)
muut_0_sõnade_arv = len(sõnad)
print(f"Sõnade arv juurde lisatud: {muut_0_sõnade_arv}")

Sõnade arv juurde lisatud: 1509


In [ ]:
# Ülejäänud muuttüüpidega andmestik
with open('muuttüübid_andmed.csv', 'r', encoding='utf-8') as muut_f:
  next(muut_f) # Ignoreerib pealise
  for rida in muut_f:
    info = rida.strip().replace('"','').split(',')
    sõna = info[0]
    sõnaliik = []
    muuttüüp = []
    # Info kättesaamine
    for i in info[1:]:
      # Sõnaliigid
      if i.isalpha():
        sõnaliik.append(i)
        continue
      # Muuttüübid
      if i.isdecimal():
        muuttüüp.append(i)
        continue
    # Mitme sõnaga tegusõnadest tegusõna eraldamine
    #if len(sõna.split()) > 1 and 'V' in sõnaliik:
    #  s = sõna.split()
    #  if s[-1] not in sõnad: # ja selle lisamine andmestikku (kui pole veel)
    #    lisa_andmestikku(s[-1], sõnaliik, muuttüüp, s[-1])
    #else:
    lisa_andmestikku(sõna, sõnaliik, muuttüüp, sõna)
muut_sõnade_arv = len(sõnad) - muut_0_sõnade_arv
print(f"Sõnade arv juurde lisatud: {muut_sõnade_arv}")

Sõnade arv juurde lisatud: 73775


Järgmise kahe koodiga analüüsitakse veelkord andmestikku, et oskaks paremini arvestada erinditega või keerulisemate üksustega

In [ ]:
algvormid_unikaalsed, count_unikaalsed = np.unique(algvormid, return_counts = True)
count_v = 0
count_mlt = 0
count_ühed = 0
for sõna, sõnaliik, muuttüüp, algvorm in zip(sõnad, sõnaliigid, muuttüübid, algvormid):
  if len(sõnaliik) > 1 and len(muuttüüp) < 2 or len(sõnaliik) < 2 and len(muuttüüp) > 1:
    count_ühed += 1
    print(sõna, sõnaliik, muuttüüp, algvorm)
  if algvorm == "--":
    print(sõna, sõnaliik, muuttüüp, algvorm)
  if len(sõna.split()) > 1:
    count_mlt += 1
  if len(sõna.split()) > 1 and 'V' in sõnaliik:
    count_v += 1

print()
print(f"Mitme sõnaga read: {count_mlt}")
print(f"Mitme sõnaga tegusõnad: {count_v}")
print(f"Sõnade arv: {len(sõnad)}")
print(f"Unikaalsed algvormid: {len(algvormid_unikaalsed)}")
print(f"Korduvad algvormid: {len(algvormid_unikaalsed[count_unikaalsed > 1])}")
print(f"Sõnade arv ja unikaalsete algvormide arvu erinevus: {len(sõnad) - len(algvormid_unikaalsed)}")
print(f"Ridade arv sõnastikus: {ridade_arv}")
print(f"Muuttüüp 0 üksuste juurde lisamine muuttüübid_andmed.csv andmestikku: {muut_sõnade_arv + 54}")
print(f"Erineva muuttüübi ja sõnaliigi arvuga : {count_ühed}")

aamen ['S', 'I'] ['2'] aamen
allapoole ['D', 'K'] [] allapoole
allpool ['D', 'K'] [] allpool
altpoolt ['D', 'K'] [] altpoolt
eespool ['D', 'K'] [] eespool
eestpoolt ['D', 'K'] [] eestpoolt
ettepoole ['D', 'K'] [] ettepoole
halleluuja ['S', 'I'] ['1'] halleluuja
heaks ['D', 'K'] [] heaks
hommikupoole ['D', 'K'] [] hommikupoole
issand ['S', 'I'] ['2'] issand
kannult ['K', 'D'] [] kannult
kilks ['S', 'I'] ['22'] kilks
kiuks ['S', 'I'] ['22'] kiuks
klõks ['S', 'I'] ['22'] klõks
klõmm ['S', 'I'] ['22'] klõmm
klõps ['S', 'I'] ['22'] klõps
kohaselt ['D', 'K'] [] kohaselt
kolks ['S', 'I'] ['22'] kolks
kraaks ['S', 'I'] ['22'] kraaks
kraks ['S', 'I'] ['22'] kraks
kraps ['S', 'I'] ['22'] kraps
kriiks ['S', 'I'] ['22'] kriiks
kriuks ['S', 'I'] ['22'] kriuks
krooks ['S', 'I'] ['22'] krooks
krõks ['S', 'I'] ['22'] krõks
krõps ['S', 'I'] ['22'] krõps
kräuks ['S', 'I'] ['22'] kräuks
kuhupoole ['D', 'K'] [] kuhupoole
kuramus ['S', 'I'] ['11'] kuramus
kurask ['S', 'I'] ['2'] kurask
kurat ['S', 'I'] ['2

In [ ]:
display(algvormid_unikaalsed[count_unikaalsed > 1])

array(['Internet', 'Interpol', 'Jane', 'Jullinen', 'Jüri', 'Kirov',
       'Margot', 'Motors', 'Ndjamena', 'Niue', 'Pavlov', 'Peedu',
       'Pradesh', 'Toomas', 'Tooming', 'Town', 'aaria', 'agar',
       'ajalehed-ajakirjad', 'alge', 'algul', 'all', 'alla',
       'allahinnatud', 'allakirjutanud', 'allakirjutatud', 'alt', 'anne',
       'arm', 'asemele', 'askleepias', 'ass', 'aste', 'bakk', 'dbase',
       'dopamiin', 'eakas', 'edasi', 'eel', 'eeltoodud', 'ees', 'eest',
       'egas', 'ehe', 'ehtima', 'eikeegi', 'eimiski', 'einoh', 'eks',
       'eksprompt', 'elamine-olemine', 'elukogenud', 'ematu-isatu',
       'emb-kumb', 'enam', 'enne', 'ennem', 'ette', 'ettesirutatud',
       'filmižanr', 'fitness', 'goot', 'haare', 'haisev', 'hakk', 'hall',
       'hambusse', 'hang', 'hange', 'hanguma', 'harvem', 'hatt',
       'hauduma', 'haue', 'heide', 'hiir', 'hirmujutt', 'hirmus',
       'hommikupäike', 'hoolimata', 'hulgas', 'hurraa', 'härras', 'hõõre',
       'hüva', 'idu', 'iga', 'ihnus',

#### Katsetused

Selles peatükis katsetatakse Vabamorfi süntesaatorit ja algoritmide loomist kõikide muutevormide genereerimiseks

In [ ]:
estnltk.vabamorf.morf.synthesize("Uus-Guinea", 'sg p', partofspeech='H')

['Uus-Guinead']

In [ ]:
estnltk.vabamorf.morf.synthesize("New York", 'sg p')

[]

In [ ]:
synthesize_all_visualize("Coca-Cola", 'N')

,case,singular,plural
0,nimetav,Coca-Cola,
1,omastav,Coca-Cola,
2,osastav,Coca-Colat,
3,lühike sisseütlev,,
4,sisseütlev,Coca-Colasse,
5,seesütlev,Coca-Colas,
6,seestütlev,Coca-Colast,
7,alaleütlev,Coca-Colale,
8,alalütlev,Coca-Colal,
9,alaltütlev,Coca-Colalt,


In [ ]:
synthesize_all_visualize("kiiremaks minema", "V")

,case,word
0,b,
1,d,
2,da,
3,des,
4,ge,
5,gem,
6,gu,
7,ks,
8,ksid,
9,ksime,


In [ ]:
synthesize_all_visualize("ainus", 'N')

,case,singular,plural
0,nimetav,"ainus, ainus","ainsad, ainsad"
1,omastav,"ainsa, ainsa","ainsate, ainsate"
2,osastav,"ainsat, ainsat, ainust, ainust","ainsaid, ainsaid"
3,lühike sisseütlev,,
4,sisseütlev,"ainsasse, ainsasse","ainsaisse, ainsaisse, ainsatesse, ainsatesse"
5,seesütlev,"ainsas, ainsas","ainsais, ainsais, ainsates, ainsates"
6,seestütlev,"ainsast, ainsast","ainsaist, ainsaist, ainsatest, ainsatest"
7,alaleütlev,"ainsale, ainsale","ainsaile, ainsaile, ainsatele, ainsatele"
8,alalütlev,"ainsal, ainsal","ainsail, ainsail, ainsatel, ainsatel"
9,alaltütlev,"ainsalt, ainsalt","ainsailt, ainsailt, ainsatelt, ainsatelt"


In [ ]:
synthesize_all_visualize("sulgema", 'V')

,case,word
0,b,"suleb, sulgeb"
1,d,"suled, sulged"
2,da,sulgeda
3,des,sulgedes
4,ge,sulgege
5,gem,sulgegem
6,gu,sulgegu
7,ks,"suleks, sulgeks"
8,ksid,"suleksid, sulgeksid"
9,ksime,"suleksime, sulgeksime"


In [ ]:
estnltk.vabamorf.morf.synthesize("k-s", 'sg n', partofspeech='Sm')

[]

In [ ]:
# Reaotsing kõik peale muuttüübi 0 andmestikus
reaotsing = 39297
with open('muuttüübid_andmed.csv', 'r', encoding='utf-8') as muut_f:
  next(muut_f) # Ignoreerib pealise
  count = 2
  for rida in muut_f:
    info = rida.strip().replace('"','').split(',')
    sõna = info[0]
    sõnaliik = []
    muuttüüp = []
    # Info kättesaamine
    for i in info[1:]:
      # Sõnaliigid
      if i.isalpha():
        sõnaliik.append(i)
        continue
      # Muuttüübid
      if i.isdecimal():
        muuttüüp.append(i)
        continue
    if count == reaotsing:
      print(sõna, sõnaliik, muuttüüp)
      break
    else:
      count += 1

New York  ['H'] ['22']


In [ ]:
ainsus, mitmus = synthesize_all("aabitsatarkus", 'N')
print(ainsus)
print(mitmus)
for sõnu in ainsus:
  if sõnu:
    print(sõnu)
    for s in sõnu:
      print(s.strip())
for sõnu in mitmus:
  if sõnu:
    print(sõnu)
    for s in sõnu:
      print(s.strip())

[['aabitsatarkus'], ['aabitsatarkuse'], ['aabitsatarkust'], [], ['aabitsatarkusesse'], ['aabitsatarkuses'], ['aabitsatarkusest'], ['aabitsatarkusele'], ['aabitsatarkusel'], ['aabitsatarkuselt'], ['aabitsatarkuseks'], ['aabitsatarkuseni'], ['aabitsatarkusena'], ['aabitsatarkuseta'], ['aabitsatarkusega']]
[['aabitsatarkused'], ['aabitsatarkuste'], ['aabitsatarkusi', 'aabitsatarkuseid'], [], ['aabitsatarkusisse', 'aabitsatarkustesse', 'aabitsatarkuseisse'], ['aabitsatarkusis', 'aabitsatarkustes', 'aabitsatarkuseis'], ['aabitsatarkusist', 'aabitsatarkustest', 'aabitsatarkuseist'], ['aabitsatarkusile', 'aabitsatarkustele', 'aabitsatarkuseile'], ['aabitsatarkusil', 'aabitsatarkustel', 'aabitsatarkuseil'], ['aabitsatarkusilt', 'aabitsatarkustelt', 'aabitsatarkuseilt'], ['aabitsatarkusiks', 'aabitsatarkusteks', 'aabitsatarkuseiks'], ['aabitsatarkusteni'], ['aabitsatarkustena'], ['aabitsatarkusteta'], ['aabitsatarkustega']]
['aabitsatarkus']
aabitsatarkus
['aabitsatarkuse']
aabitsatarkuse
['aab

In [ ]:
vormid = synthesize_all("blokaadi sulgema", 'V')
print(vormid)
for sõnu in vormid:
  if sõnu:
    print(sõnu)
    for s in sõnu:
      print(s)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [ ]:
with open('muuttüübid_andmed.csv', 'r', encoding='utf-8') as muut_f:
  next(muut_f) # Ignoreerib pealise
  for rida in muut_f:
    info = rida.strip().replace('"','').split(',')
    sõna = info[0]
    sõnaliik = []
    muuttüüp = []
    # Info kättesaamine
    for i in info[1:]:
      # Sõnaliigid
      if i.isalpha():
        sõnaliik.append(i)
        continue
      # Muuttüübid
      if i.isdecimal():
        muuttüüp.append(i)
        continue
    if muuttüüp == ['3', '5']:
      print(info)
      print()

['ainus', 'S', 'S', '3', '5']

['armas', 'A', 'A', '3', '5']

['ebasünnis', 'A', 'A', '3', '5']

['ergas', 'A', 'A', '3', '5']

['hoolas', 'A', 'A', '3', '5']

['hõlbus', 'A', 'A', '3', '5']

['ihnus', 'A', 'A', '3', '5']

['imearmas', 'A', 'A', '3', '5']

['kõikvõimas', 'A', 'A', '3', '5']

['mõjuvõimas', 'A', 'A', '3', '5']

['mõtteergas', 'A', 'A', '3', '5']

['siivas', 'A', 'A', '3', '5']

['sünnis', 'A', 'A', '3', '5']

['tahtevõimas', 'A', 'A', '3', '5']

['teovõimas', 'A', 'A', '3', '5']

['tõrges', 'A', 'A', '3', '5']

['valvas', 'A', 'A', '3', '5']

['voolas', 'A', 'A', '3', '5']

['võimas', 'A', 'A', '3', '5']

['õilis', 'A', 'A', '3', '5']

['õnnis', 'A', 'A', '3', '5']

['õõnes', 'A', 'A', '3', '5']

['ärgas', 'A', 'A', '3', '5']

['üksainus', 'S', 'S', '3', '5']

['ülivõimas', 'A', 'A', '3', '5']



#### Reeglid

Selles peatükis on loodud reeglistik ühe ja mitme muuttüübiga sünteesitud muutevormide kogumiseks

In [ ]:
sõnad = list()
sõnaliigid = list()
muuttüübid = list()
algvormid = list()

sünteesimata_sõnad = list()

*Kohati kontrollimine tähendab, et andmestikust on võetud suvaliselt 5 vastava muuttüübiga kirjed*

**Käändsõnad mitme muuttüübiga**

* [11, 9] kohati kontrollimisel korras,
  * kui on kolme valikuga vormid, siis
    * 11 on -i lõpuga (esimene), mitmuse osastava käände järgi saab kätte lõpu
    * -te lõpp on mõlemal (teine) omastav kääne + käändelõpp,
    * 9 on -ei (kolmas), mitmuse osastava käände järgi saab kätte lõpu

* [12, 10] kohati kontrollimisel korras
  * kui on kolme valikuga vormid, siis
    * 12 on -i lõpuga (esimene), mitmuse osastava käände järgi saab kätte lõpu
    * -te lõpp on mõlemal (teine) omastav kääne + käändelõpp,
    * 10 on -ei (kolmas), mitmuse osastava käände järgi saab kätte lõpu

* [23, 22] kohati kontrollimisel korras
  * kui on kahe valikuga vormid, siis
    * 23 on -te lõpuga (esimene), omastav kääne + käändelõpp
    * 22 on -de (teine), omastav kääne + käändelõpp
  * on olemas veel vorme, mis on tuletatud lühikesest mitmuse osastavast käändest, mõlemal on need vormid

* [19, 2] kohati kontrollimisel korras
  * mitmuse osastava -sid lõpuga välja jätta (viimane järjendis)
  * kui on nelja valikuga vormid, siis
    * 19 on -de ja -e lõpuga, tuletatavad vastavalt mitmuse omastava ja osastava käändest (esimene ja kolmas)
    * 2 on -te ja -ei lõpuga, tuletatavad vastavalt mitmuse omastava ja osastava käändest (teine ja neljas)

* [3, 5] tuleb käsitsi läbi käia
  * Mõnel tuleb muuttüübi 3 muutevorm enne, mõnel muuttüübi 5 muutevorm
  * Tuleb käsitsi üle vaadata, sest emb-kumb number võib olla esimene
  * Mitmuse vormid on mõlemal samad, aga ainsuse vorm on erinev

* [10, 12] üle vaja kontrollida
  * Mõned sõnavormid ei eksisteeri Sõnaveebi ühendsõnastikus
  * Mitmuse osastav -ei lõpuga (kolmas) ja omastav (teine), -i lõpuga mitmuse vormid välja jätta (esimene)

* [5, 7] õiget pidi
  * -ei lõpuga mitmuse vormid mõlemal, mitmuse osastavast tuletatav (esimene)
  * 5 -te lõpuga mitmuse vormid, mitmuse esimese omastavast tuletatav (teine)
  * 7 -ste lõpuga mitmuse vormid, mitmuse teise omastavast tuletatav (kolmas)

* [22, 22] viga sõnastikus, rasedustest topelt: esimesel ühe muuttüübiga (22), teisel kahe samasuguse muuttüübiga (22, 22) -- selle sõna jätab välja andmestikust


**Pöördsõnad mitme muuttüübiga**

* [31, 27] kohati kontrollimisel korras
  * Mitme valiku puhul:
    * 31 on järjendi esimene element
    * 27 on järjendi teine element

* [28, 27] kohati kontrollimisel korras
  * Mitme valiku puhul:
    * 28 on järjendi esimene element
    * 27 on järjendi teine element

In [ ]:
# Rööptüüpidele 3,5 ja 5,3 sünteesitakse erinevates järjekordades muutevormid, kuid kirjetes on rööptüüpides esinevad muuttüübid sama järjestusega
count3_5_vastused = [3, 5, 3, 5, 5, 5, 3, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 3, 5]
count3_5 = 0

# Sünteesimata sõnade loendamiseks
sünteesimata_arv = 0

def käsitle_ühe_muuttüübiga(sõna, sõnaliik, muuttüüp):
  global sõnad, sõnaliigid, muuttüübid, sünteesimata_arv, sünteesimata_sõnad

  sünteesimata = True

  if pos[sõnaliik[0]] == 'N':

    ainsus, mitmus = synthesize_all(sõna, pos[sõnaliik[0]])

    for sõnu in ainsus:
      if sõnu:
        sünteesimata = False
        for s in sõnu: # Võivad tekkida duplikaadid
          lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

    for sõnu in mitmus:
      if sõnu:
        sünteesimata = False
        for s in sõnu: # Võivad tekkida duplikaadid
          lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

    if sünteesimata:
      sünteesimata_arv += 1
      sünteesimata_sõnad.append(sõna)

  else:

    vormid = synthesize_all(sõna, pos[sõnaliik[0]])

    for sõnu in vormid:
      if sõnu:
        sünteesimata = False
        for s in sõnu: # Võivad tekkida duplikaadid
          lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

    if sünteesimata:
      sünteesimata_arv += 1
      sünteesimata_sõnad.append(sõna)

def käsitle_mitme_muuttüübiga(sõna, sõnaliik, muuttüüp):
  global sõnad, sõnaliigid, muuttüübid, count3_5, sünteesimata_arv, sünteesimata_sõnad

  sünteesimata = True

  if pos[sõnaliik[0]] == 'N':
    ainsus, mitmus = synthesize_all(sõna, pos[sõnaliik[0]])

    if muuttüüp == ['3', '5']:
      vastus = count3_5_vastused[count3_5]
      count3_5 += 1
      #display(synthesize_all_visualize(sõna, pos[sõnaliik[0]]))
      #vastus = None
      #while vastus != 3 and vastus != 5:
      #  vastus = int(input("Kumb muuttüüp on eespool? Sisesta muuttüübi number (3 või 5): "))

    osastav_pandud = False

    for i in range(15): # Käänete kaupa (14 käänet + lühike sisseütlev)
      sõnad_ainsus = ainsus[i]
      sõnad_mitmus = mitmus[i]

      match muuttüüp:

        case ['11', '9']:

          if sõnad_ainsus:
            sünteesimata = False
            for s in sõnad_ainsus: # Võivad tekkida duplikaadid
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

          if sõnad_mitmus:
            sünteesimata = False

            if len(sõnad_mitmus) == 2: # Mitmuse osastav
              for j in range(len(sõnad_mitmus)):
                lisa_andmestikku(sõnad_mitmus[j], [sõnaliik[j]], [muuttüüp[j]], sõna)

            elif len(sõnad_mitmus) == 3: # Mitmuse vormid
              lisa_andmestikku(sõnad_mitmus[0], [sõnaliik[0]], [muuttüüp[0]], sõna)
              lisa_andmestikku(sõnad_mitmus[1], sõnaliik, muuttüüp, sõna)
              lisa_andmestikku(sõnad_mitmus[2], [sõnaliik[1]], [muuttüüp[1]], sõna)

            else: # Ühe sõnaga vormid
              for s in sõnad_mitmus:
                lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

        case ['12', '10']:

          if sõnad_ainsus:
            sünteesimata = False
            for s in sõnad_ainsus: # Võivad tekkida duplikaadid
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

          if sõnad_mitmus:
            sünteesimata = False

            if len(sõnad_mitmus) == 2: # Mitmuse osastav
              for j in range(len(sõnad_mitmus)):
                lisa_andmestikku(sõnad_mitmus[j], [sõnaliik[j]], [muuttüüp[j]], sõna)

            elif len(sõnad_mitmus) == 3: # Mitmuse osastavast järgmised
              lisa_andmestikku(sõnad_mitmus[0], [sõnaliik[0]], [muuttüüp[0]], sõna)
              lisa_andmestikku(sõnad_mitmus[1], sõnaliik, muuttüüp, sõna)
              lisa_andmestikku(sõnad_mitmus[2], [sõnaliik[1]], [muuttüüp[1]], sõna)

            else: # Ühe sõnaga vormid
              for s in sõnad_mitmus:
                lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

        case ['23', '22']:

          if sõnad_ainsus:
            sünteesimata = False
            for s in sõnad_ainsus: # Võivad tekkida duplikaadid
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

          if sõnad_mitmus:
            sünteesimata = False

            if len(sõnad_mitmus) == 2: # Mitmuse omastav ja teised
              for j in range(len(sõnad_mitmus)):
                lisa_andmestikku(sõnad_mitmus[j], [sõnaliik[j]], [muuttüüp[j]], sõna)

            else: # Ühe sõnaga vormid
              for s in sõnad_mitmus:
                lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

        case ['19', '2']:

          if sõnad_ainsus:
            sünteesimata = False
            for s in sõnad_ainsus: # Võivad tekkida duplikaadid
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

          if sõnad_mitmus:
            sünteesimata = False

            if len(sõnad_mitmus) == 3: # Mitmuse omastav
              for j in range(len(sõnad_mitmus)-1): # Viimast mitte võtta
                lisa_andmestikku(sõnad_mitmus[j], [sõnaliik[j]], [muuttüüp[j]], sõna)

            elif len(sõnad_mitmus) == 4: # Mitmuse osastavast järgmised
              for j in range(len(sõnad_mitmus)):
                lisa_andmestikku(sõnad_mitmus[j], [sõnaliik[j%2]], [muuttüüp[j%2]], sõna) # % saab 0 ja 1

            else: # Ühe ja kahe sõnaga vormid
              for s in sõnad_mitmus:
                lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

        case ['3', '5']:

          if sõnad_ainsus:
            sünteesimata = False

            if len(sõnad_ainsus) == 2:

              if vastus == 3:
                lisa_andmestikku(sõnad_ainsus[0], [sõnaliik[0]], [muuttüüp[0]], sõna)
                lisa_andmestikku(sõnad_ainsus[1], [sõnaliik[1]], [muuttüüp[1]], sõna)

              else:
                lisa_andmestikku(sõnad_ainsus[0], [sõnaliik[1]], [muuttüüp[1]], sõna)
                lisa_andmestikku(sõnad_ainsus[1], [sõnaliik[0]], [muuttüüp[0]], sõna)

            elif len(sõnad_ainsus) == 6: # "üksainus" ainsuse osastav
              lisa_andmestikku(sõnad_ainsus[1], [sõnaliik[0]], [muuttüüp[0]], sõna)

            else:
              for s in sõnad_ainsus: # Võivad tekkida duplikaadid
                lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

          if sõnad_mitmus:
            sünteesimata = False
            for s in sõnad_mitmus:
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

        case ['10', '12']:

          if sõnad_ainsus:
            sünteesimata = False
            for s in sõnad_ainsus: # Võivad tekkida duplikaadid
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

          if sõnad_mitmus:
            sünteesimata = False

            if len(sõnad_mitmus) == 2: # Mitmuse osastav ja ni-na-ta-ga vormid

              if not osastav_pandud: # Mitmuse osastav
                lisa_andmestikku(sõnad_mitmus[1], [sõnaliik[1]], [muuttüüp[1]], sõna)

              else: # ni-na-ta-ga vormid
                lisa_andmestikku(sõnad_mitmus[0], [sõnaliik[0]], [muuttüüp[0]], sõna)

            if len(sõnad_mitmus) == 3: # Mitmuse osastavast järgmised
              for j in range(1, len(sõnad_mitmus)): # Esimest mitte võtta
                lisa_andmestikku(sõnad_mitmus[j], [sõnaliik[j-1]], [muuttüüp[j-1]], sõna)

            else:
              for s in sõnad_mitmus:
                lisa_andmestikku(s, [sõnaliik[0]], [muuttüüp[0]], sõna)

        case ['5', '7']:

          if sõnad_ainsus:
            sünteesimata = False
            for s in sõnad_ainsus: # Võivad tekkida duplikaadid
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

          if sõnad_mitmus:
            sünteesimata = False

            if len(sõnad_mitmus) == 2: # Mitmuse omastav ja ni-na-ta-ga'st mõned
              for j in range(len(sõnad_mitmus)):
                lisa_andmestikku(sõnad_mitmus[j], [sõnaliik[j]], [muuttüüp[j]], sõna)

            elif len(sõnad_mitmus) == 3: # Mitmuse osastavast järgmised
              lisa_andmestikku(sõnad_mitmus[0], [sõnaliik[0]], [muuttüüp[0]], sõna)
              lisa_andmestikku(sõnad_mitmus[1], sõnaliik, muuttüüp, sõna)
              lisa_andmestikku(sõnad_mitmus[2], [sõnaliik[1]], [muuttüüp[1]], sõna)

            else: # Ühe sõnaga vormid
              for s in sõnad_mitmus:
                lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

    if sünteesimata:
      sünteesimata_arv += 1
      sünteesimata_sõnad.append(sõna)

  else:
    vormid = synthesize_all(sõna, pos[sõnaliik[0]])

    if not vormid:
      sünteesimata_arv += 1
      sünteesimata_sõnad.append(sõna)

    match muuttüüp:

      case ['31', '27']:
        for sõnu in vormid:

          if len(sõnu) == 2:
            for j in range(len(sõnu)):
              lisa_andmestikku(sõnu[j], [sõnaliik[j]], [muuttüüp[j]], sõna)

          else:
            for s in sõnu:
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

      case ['28', '27']:
        for sõnu in vormid:

          if len(sõnu) == 2:
            for j in range(len(sõnu)):
              lisa_andmestikku(sõnu[j], [sõnaliik[j]], [muuttüüp[j]], sõna)

          else:
            for s in sõnu:
              lisa_andmestikku(s, sõnaliik, muuttüüp, sõna)

### Süntees

Selles peatükis sünteesitakse algvormidele kõik muutevormid.

0 muuttüüpidel on kõik muutevormid juba olemas, need lisatakse otse uude andmestiku.

In [ ]:
with open('muuttüübi_0_andmed.csv', 'r', encoding='utf-8') as muut_0_f:
  next(muut_0_f)
  for rida in muut_0_f:
    info = rida.strip().replace('"','').split(',')
    sõna = info[0]
    sõnaliik = []
    muuttüüp = []
    algvorm = info[-1]
    for i in info[1:-1]:
      if i.isalpha():
        sõnaliik.append(i)
        continue
      if i.isdecimal():
        muuttüüp.append(i)
        continue
    if len(sõnaliik) > 1 or len(muuttüüp) > 1: # Kontrolliks
      print(rida)
      print("Mitme sõnaliigi või muuttüübiga")
    lisa_andmestikku(sõna, sõnaliik, muuttüüp, algvorm)

Ülejäänud muuttüüpidele genereeritakse ning kogutakse muutevorme kokku vastavalt koostatud reeglistikule.

Lisaks, ei arvestada tegusõnadeühendi kirjeid ehk sõnaühendeid, mis koosnevad lisaks tegusõnale veel täpsustavast abimäärsõnast, näiteks "alla andma". Välja arvatud juhul kui tegusõna ise pole andmestikus olemas.

In [ ]:
with open('muuttüübid_andmed.csv', 'r', encoding='utf-8') as muut_f:
  next(muut_f) # Ignoreerib pealise
  for rida in muut_f:
    info = rida.strip().replace('"','').split(',')
    sõna = info[0]
    sõnaliik = []
    muuttüüp = []
    # Info kättesaamine
    for i in info[1:]:
      # Sõnaliigid
      if i.isalpha():
        sõnaliik.append(i)
        continue
      # Muuttüübid
      if i.isdecimal():
        muuttüüp.append(i)
        continue
    # Tegusõnaühenditest tegusõna eraldamine
    if len(sõna.split()) > 1 and 'V' in sõnaliik:
      s = sõna.split()
      if s[-1] not in sõnad: # ja selle lisamine andmestikku (kui pole veel)
        if len(muuttüüp) == 1:
          käsitle_ühe_muuttüübiga(s[-1], sõnaliik, muuttüüp)
        else:
          käsitle_mitme_muuttüübiga(s[-1], sõnaliik, muuttüüp)
      else:
        sünteesimata_arv += 1
        sünteesimata_sõnad.append(sõna)

    # Pole muuttüüpi
    elif len(muuttüüp) == 0:
      if len(sõnaliik) == 0: # ega sõnaliiki
        lisa_andmestikku(sõna, ["-"], ["-"], sõna)
      else:
        lisa_andmestikku(sõna, sõnaliik, ["-"], sõna)
    # Ühe muuttüübiga
    elif len(muuttüüp) == 1:
      käsitle_ühe_muuttüübiga(sõna, sõnaliik, muuttüüp)
    # Mitme muuttüübiga
    else:
      käsitle_mitme_muuttüübiga(sõna, sõnaliik, muuttüüp)

In [ ]:
print(sünteesimata_arv)
display(sünteesimata_sõnad)

142


['Abu Dhabi',
 'Addis Abeba',
 'Al Gore',
 'allpool olev',
 'Andhra Pradesh',
 'Andorra la Vella',
 'Bab el-Mandebi',
 'Babia Gora',
 'Baltic Fair',
 'Baltic News Service',
 'Balti meri',
 'Bandar Seri Begawan',
 'Buenos Aires',
 'Burkina Faso',
 'Cabo Verde',
 'Charlotte Amalie',
 'Ciudad de México',
 'Ciudad de Mexico',
 'Coca Cola',
 'Cockburn Town',
 'Costa Rica',
 'Côte dIvoire',
 'Cote dIvoire',
 'Dagens Nyheter',
 'Dar es Salaam',
 'Dar es Salam',
 'Eestimaa rahvas',
 'eesti rahvas',
 'El Salvador',
 'ema vend',
 'ema õde',
 'enesele lõputegemine',
 'Estonian Air',
 'Express Hotline',
 'Flying Fish Cove',
 'Fort-de France',
 'Franz Joseph',
 'General Motors',
 'Ha Noi',
 'Ho Chi Minh',
 'Home Island',
 'Hovery Covery',
 'ignoma',
 'Ile-de France',
 'Intellimouse',
 'isa vend',
 'isa õde',
 'Jan Mayen',
 'järeltehtu',
 'Karlovy Vary',
 'Krivoi Rog',
 'k-s',
 'Kuala Lumpur',
 'La Habana',
 'La Manche',
 'Las Palmas',
 'Las Vegas',
 'Le Coq',
 'lendu laskmine',
 'Lietuvos rytas',
 

### Pandase *dataframe*'i koostamine

Selles peatükis koostatakse Pandase *dataframe*, mis koosneb algvormidest ja sünteesitud muutevormidest.

In [ ]:
# Mõõtmed
print(len(sõnad), len(sõnaliigid), len(muuttüübid), len(algvormid))

2316847 2316847 2316847 2316847


In [ ]:
# Andmetüübi muutmine
df = pd.DataFrame({'Sõnad': sõnad, 'Sõnaliigid': sõnaliigid, 'Muuttüübid': muuttüübid, 'Algvormid': algvormid})
df['Sõnaliigid'] = df['Sõnaliigid'].apply(str)
df['Muuttüübid'] = df['Muuttüübid'].apply(str)

In [ ]:
display(df)

,Sõnad,Sõnaliigid,Muuttüübid,Algvormid
0,ajalehed-ajakirjad,['S'],['0'],ajalehed-ajakirjad
1,ajalehtede-ajakirjade,['S'],['0'],ajalehed-ajakirjad
2,ajalehtesid-ajakirjasid,['S'],['0'],ajalehed-ajakirjad
3,ajalehti-ajakirju,['S'],['0'],ajalehed-ajakirjad
4,ajalehtedesse-ajakirjadesse,['S'],['0'],ajalehed-ajakirjad
...,...,...,...,...
2316842,Ypresiks,['H'],['2'],Ypres
2316843,Ypresini,['H'],['2'],Ypres
2316844,Ypresina,['H'],['2'],Ypres
2316845,Ypresita,['H'],['2'],Ypres


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2316847 entries, 0 to 2316846
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Sõnad       object
 1   Sõnaliigid  object
 2   Muuttüübid  object
 3   Algvormid   object
dtypes: object(4)
memory usage: 70.7+ MB


In [ ]:
# NaN või NA väärtused andmestikus
null_values = df.isnull().any()
print(null_values)

Sõnad         False
Sõnaliigid    False
Muuttüübid    False
Algvormid     False
dtype: bool


In [ ]:
# Kontrolliks
list_sõnad = df[df['Sõnad'].str.contains(',')]
list_sonaliigid = df[df['Sõnaliigid'].isnull()]
list_muuttuubid = df[df['Muuttüübid'].isnull()]
list_algvormid = df[df['Algvormid'].str.contains(',')]
list_mitusõna = df[df['Sõnad'].str.contains(' ')]

display(list_sõnad)
display(list_sonaliigid)
display(list_muuttuubid)
display(list_algvormid)
display(list_mitusõna)

,Sõnad,Sõnaliigid,Muuttüübid,Algvormid


,Sõnad,Sõnaliigid,Muuttüübid,Algvormid


,Sõnad,Sõnaliigid,Muuttüübid,Algvormid


,Sõnad,Sõnaliigid,Muuttüübid,Algvormid


,Sõnad,Sõnaliigid,Muuttüübid,Algvormid
37534,á la carte,['D'],['-'],á la carte
97391,asi ants,['D'],['-'],asi ants
110657,au pair,['D'],['-'],au pair
154993,de facto,['D'],['-'],de facto
156800,de jure,['D'],['-'],de jure
213452,eilsel päeval,['D'],['-'],eilsel päeval
217647,eksituse tõttu,['D'],['-'],eksituse tõttu
352946,head aega,['I'],['-'],head aega
357960,heitliku meelega,['D'],['-'],heitliku meelega
428930,iga kord,['D'],['-'],iga kord


In [ ]:
# Duplikaatide eemaldamine
df_puhas = df.drop_duplicates(subset=['Sõnad', 'Sõnaliigid', 'Muuttüübid', 'Algvormid'])
display(df_puhas)
print(df.shape, "->", df_puhas.shape)

display(df_puhas)

,Sõnad,Sõnaliigid,Muuttüübid,Algvormid
0,ajalehed-ajakirjad,['S'],['0'],ajalehed-ajakirjad
1,ajalehtede-ajakirjade,['S'],['0'],ajalehed-ajakirjad
2,ajalehtesid-ajakirjasid,['S'],['0'],ajalehed-ajakirjad
3,ajalehti-ajakirju,['S'],['0'],ajalehed-ajakirjad
4,ajalehtedesse-ajakirjadesse,['S'],['0'],ajalehed-ajakirjad
...,...,...,...,...
2316842,Ypresiks,['H'],['2'],Ypres
2316843,Ypresini,['H'],['2'],Ypres
2316844,Ypresina,['H'],['2'],Ypres
2316845,Ypresita,['H'],['2'],Ypres


(2316847, 4) -> (2240515, 4)


,Sõnad,Sõnaliigid,Muuttüübid,Algvormid
0,ajalehed-ajakirjad,['S'],['0'],ajalehed-ajakirjad
1,ajalehtede-ajakirjade,['S'],['0'],ajalehed-ajakirjad
2,ajalehtesid-ajakirjasid,['S'],['0'],ajalehed-ajakirjad
3,ajalehti-ajakirju,['S'],['0'],ajalehed-ajakirjad
4,ajalehtedesse-ajakirjadesse,['S'],['0'],ajalehed-ajakirjad
...,...,...,...,...
2316842,Ypresiks,['H'],['2'],Ypres
2316843,Ypresini,['H'],['2'],Ypres
2316844,Ypresina,['H'],['2'],Ypres
2316845,Ypresita,['H'],['2'],Ypres


In [ ]:
#df.to_csv('andmed.csv', index=False)
df_puhas.to_csv('andmed_algvormidega.csv', index=False)

#### Andmestiku kontrollimine

In [ ]:
df_algvormidega = pd.read_csv("andmed_algvormidega.csv", header=0, keep_default_na=False)

In [ ]:
list_nimed = df_algvormidega[df_algvormidega['Sõnaliigid'].str.contains('H')]
display(list_nimed)

,Sõnad,Sõnaliigid,Muuttüübid,Algvormid
1623,Aabel,H,2,Aabel
1624,Aabeli,H,2,Aabel
1625,Aabelit,H,2,Aabel
1626,Aabelisse,H,2,Aabel
1627,Aabelis,H,2,Aabel
...,...,...,...,...
2240510,Ypresiks,H,2,Ypres
2240511,Ypresini,H,2,Ypres
2240512,Ypresina,H,2,Ypres
2240513,Ypresita,H,2,Ypres


In [ ]:
# Algvormide kaupa väärtuste loendamine
algvorm_counts = list_nimed['Algvormid'].value_counts()
display(algvorm_counts)

Toomas         116
Niue           102
Jullinen       100
Margot         100
Atlas           88
              ... 
Londonderry     13
Alatskivi       12
Laptevite       12
Anseküla        12
Raikküla        12
Name: Algvormid, Length: 3644, dtype: int64

In [ ]:
# Otsing
otsing = 'New York'
vasted = list_nimed[list_nimed['Algvormid'].str.contains(otsing)]
print(f"Read, kus 'Algvorm' sisaldab '{otsing}':")
display(vasted)

Read, kus 'Algvorm' sisaldab 'New York':


,Sõnad,Sõnaliigid,Muuttüübid,Algvormid


In [ ]:
otsing = 'pank'
vasted = df_algvormidega[df_algvormidega['Algvormid'].str.contains(otsing)]
print(f"Read, kus 'Algvorm' sisaldab '{otsing}':")
display(vasted)

Read, kus 'Algvorm' sisaldab 'pank':


,Sõnad,Sõnaliigid,Muuttüübid,Algvormid
214848,ekspankur,S,2,ekspankur
214849,ekspankuri,S,2,ekspankur
214850,ekspankurit,S,2,ekspankur
214851,ekspankurisse,S,2,ekspankur
214852,ekspankuris,S,2,ekspankur
...,...,...,...,...
2220557,ühispankuriteni,S,2,ühispankur
2220558,ühispankureina,S,2,ühispankur
2220559,ühispankuritena,S,2,ühispankur
2220560,ühispankuriteta,S,2,ühispankur
